# **Binomial Options Pricing Model** <br>
This notebook will describe the derivations to equations in the BOPM followed by the strategy from a developers standpoint for implementation. The model will be implemented in both a slow and fast way.

## **Binomial Model Derivation - Replicating Portfolio of European Call Option**
The approach I will be taking to develop this model will be via the **Replicating Portfolio Approach**.

- Here the perspective is from the seller of the European Call Option. 
- Once the option has been sold, the seller is in a short position and is exposed to potential financial liability. 
- Therefore the seller decides to hedge this short position by buying stocks (and investing in risk-free bonds). 
- If the portfolio replicates the payoff of the option at each node, the seller is considered perfectly hedged.

### Portfolio

We are going to formulate this model for a one-period binomial tree i.e. the option expires after one time period.

Let $\Delta$ be the number of shares of stock bought
Let $B$ be the amount of money invested in risk-free bonds

The value of the portfolio $C$ is given by $$C = \Delta \cdot S_0 + B$$
where $S_0$ is the initial stock price when the option was sold

Now to be completely hedged, the portfolio needs to match the option's payoff at that time depending on if the stock goes up or down.

If the stock goes up: $$\Delta \cdot S_u + Be^{r \Delta t} = C_u$$

If the stock goes down: $$\Delta \cdot S_d + Be^{r \Delta t} = C_d $$

where $S_u = S_0 \cdot u$ , $S_d = S_0 \cdot d$ , $u$ is the up-factor of the stock and $d = \frac{1}{u}$

Note that the risk free interest rate applied is the same regardless of stock movement since it is a different market.

### Solving $\Delta$ and $B$

Subtracting both equations we end up with the formula for $\Delta$: $$\Delta = \frac{C_u - C_d}{S_u - S_d}$$

Substituing this into either the stock up/down equation we obtain $$B = e^{-r \Delta t} \left( \frac{S_u C_d - S_d C_u}{S_u - S_d} \right)$$

### Calculating the Initial/Previous Option Price

Note that the previous option price is given by the portfolio value at that time: $$C = \Delta \cdot S_0 + B$$
We can substitue $\Delta$ and $B$ into this to get: $$C = \frac{C_u - C_d}{S_u - S_d} + e^{-r \Delta t} \left( \frac{S_u C_d - S_d C_u}{S_u - S_d} \right)$$ 

### Discounted Expectation Formula

This after simplifcation and substituion with variables expressed in terms of $u$ and $d$ results in the **Discounted Expectation Formula**. 

$$C = e^{-r \Delta t} [pC_u + (1-p)C_d]$$ 

where $$ p = \frac{e^{r \Delta t - d}}{u - d}$$ and $$ u = e^{\sigma \sqrt{\Delta t}} , d = \frac{1}{u}$$

This could have been calculated with a simpler probablistic approach of expected value but the approach above demonstrates how the option price is derived using a hedging portfolio to eliminate risk and still acheive arbitrage-free pricing.

## **Implementation Strategy**<br>

This section will strategise how to use the previous section on a one-period binomial model and how to expand it to calculate the fair option price given a multi-period binomial tree.

- First let's consider the binomial tree with nodes and edges. 
- It is a multi-period binomial tree (say 3 steps). 
- If we want to calculate the initial contract fair price, we can use our one-period binomial tree to work backwards from the last stage to the initial node
- Looking closely at the binomial tree, we can calculate all the previous layer's contract prices by splitting the tree into smaller one period binomial trees
- We repeat this step going backwards till we get to the first node - which is the **Fair Contract Price**

### Binomial Tree Setup

In order to start working backwards, we first need to calculate the contract prices at the terminal node layer.

Considering the BT as a tree with nodes and edges that can be indexed using $i,j$ indexing (0-indexed) where $j = 0$ is the lowest node for the particular layer and $i$ is the particular layer.

### Stock Price - Terminal Node Layer
First calculate the stock price for a node on the terminal $i = N$ layer:

$$S_{N,j} = S_0 \cdot u^j \cdot d^{N-j}$$

This formula comes from the repeated multiplication of $u$ and $d$ along the various edges from the initial stock price. This formula is valid only if the BT is recombining (ensured by $u = 1/d$)

### Contract Payoff - Terminal Node Layer
Now we can calculate the contract prices. The rationale behind this is based on 2 scenarios. 
1) If the option is exercised, the payoff is determined by $S - K$ because that is what profit the buyer will get by immediately **selling** the stock at the current price $ $S$ once bought for $ $K$. 
2) If $S$ is below the strike price $K$, the buyer will not exercise the option since the buyer will lose money selling stock at a lower than bought price

Hence we get the equation for a contract by: 

$$ C_{i,j} = max\{ S_{i,j} - K,0 \}$$

### Contract Price Formula - One Period BT

With our new notation, our new formula for the previous node's contract price is given by:

$$C_{i,j} = e^{-r \Delta t} [pC_{i+1,j+1} + (1-p)C_{i+1,j}]$$ 

Now we can use this because we can calculate the contract payoffs for the terminal nodes from the previous section!

### Backward Induction

After calculating the contract prices on the terminal layer, we use the DEF formula to calculate the previous node's contract payoffs. And then the previous nodes etc. etc. until we get to the 0th + 1 layer where we can finally calculate the initial option payoff and hence the fair option price. 

### Computationally

We will consider each layer as NumPy arrays and iterate backwards, knocking off the N-1 indexed element at each stage so that the array's relevant values reduce till the first element.

In [31]:
import numpy as np

In [32]:
# defining variables
#TODO: change u and d to account for volatility
S0 = 100    # initial stock price
K = 103     # strike price
T = 1       # time to maturity (years)
N = 4       # no. of time-steps
r = 0.06    # risk-free annual interest rate
u = 1.2     # up factor
d = 1/u     # down factor (inverse for recombining tree)
opt = 'C'   # options type (C - call ; P - put)

## **Binomial Tree - for loops (Slow)**
In this model, we will iterate through the notes using for loops which is slow. Later we will see how to speed this up using numpy vectorisation. 

The strategy is to first compute the contract values at the terminal node layer and work backwards till the initial contract value which will give us the fair options price.

In [33]:
def binomial_slow(S0,K,T,N,r,u,d,opt='C'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)

    # numpy array to store the stock values at maturity
    S = np.zeros(N+1)
    S[0] = d**N * S0
    for j in range(1,N+1):
        S[j] = S0 * (u ** j) * (d ** (N-j))

    # numpy array to store options values at maturity'
    C = np.zeros(N+1)
    for j in range(0,N+1):
        C[j] = max(0,S[j] - K)
    
    # now let's move backwards through the tree
    for i in np.arange(N,0,-1):
        for j in range(0,i):
            # replace the cell in the nparray with the backwards contract calculation of itself and the one above
            # essentially knocking off the top element            
            C[j] = discount * (p * C[j+1] + (1-p) * C[j]) 
    
    return C[0]

In [34]:
result = round(float(binomial_slow(S0,K,T,N,r,u,d,opt='C')),2)
result

15.43

## **Binomial Tree - numpy vectorisation (Fast)**

The same mathematics and techniques are going to be used here but instead of iterating using for loops we will speed up the multiplication process by using vectors.

The strategy to be used is as follows:
1) We initialise a numpy array (vector) **C** using np.arange(). This will contain the stock prices at the terminal node layer

2) Then we subtract the scalar **K** from the vector **C** and compute the maximum for each entry in the vector between $ K - C $ and the zero vector. This will satisfy this equation $$C_{i,j} = max(S_{i,j} - K, 0)$$

3) Now we step backwards through the tree from the Nth (terminal) layer to the 1st layer (not the 0th layer). In order to efficiently calculate the previous layer's contract prices we will use vector slicing. To calculate the $i-1$ node at a height $j$, we take the nodes from $i$ at positions $j$ and $j+1$ and we do this for every single node at layer $i-1$. For each node this acts as a one period binomial tree. It therefore makes sense to slice the $i$ th layer's vector from $0$ to $i-1$ (inclusive) for the node positions at $j$ and slice the $i$ th layer's vector from $1$ to $i$ (inclusive) for node positions at $j+1$ for each one period binomial tree. 

4) Once we've reached the final one period binomial tree, the initial node will have been calculated (i.e. its contract price has been calculated) so we output that



In [35]:
def binomial_fast(S0=100,K=103,T=1,N=3,r=0.06,u=1.2,d=1/u,opt='C'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)

    #initialise array with stock prices
    C = S0 * u ** (np.arange(0,N+1,1)) * d ** (np.arange(N,-1,-1))

    #calculate option payoffs using vectors
    C = np.maximum(C-K, np.zeros(N+1))

    #iterate backwards and multiply vectors so that each index in previous step corresponds
    #to the same index times the one above it - hence the slicing
    for i in np.arange(N,0,-1):
        C = discount * (p * C[1:i+1] + (1-p) * C[0:i])
    
    return C[0]

In [36]:
result = round(float(binomial_fast(S0=S0,K=K,T=T,N=N,r=r,u=u,d=d,opt='C')), 2)
C = result
C

15.43

## **Put-Call Parity Derivation**

Put-Call parity is a relationship between European Call and Put options that allows for no-arbitrage pricing.

We can derive this by creating 2 equivalent portfolios to avoid arbitrage. 

**Portfolio A**
- Buy one European Call Option ($C$)
- Sell one European Put Option ($P$)
- Have cash at discounted price ($Ke^{-rT}$)

**Portfolio B**
- Buy one share of stock ($S_0$)

There are now 2 scenarios with stock price $S_T$:

1) $S_T > K$ 
    - Exercise call option - payoff of $S_T - K$#
    - Put Option expires
    - Cash then grows to K because interest rate
    - Portfolio A Value: $S_T - K + K = S_T$
    - Portfolio B Value: $S_T$

2) $S_T < K$
    - Call Option expires
    - Put Option exercised - loss of $K - S_T$
    - Cash grows to K
    - Portfolio A Value: $K -(K - S_T) = S_T$
    - Portfolio B Value: $S_T$

Hence in both scenarios the Portfolio Values are equal. This means that the initial portfolio values must be equal due to no-arbitrage.

This leads to the equation $$C + Ke^{-rT} - P = S_0$$

## **Fairly Pricing European Put Option - Put-Call Parity & BOPM**

Now given we have calculated the fair price of a European Call Option, and we have the remaining variables, we can calculate fair price of the European Put Option using the rearranged Put-Call parity formula: 

$$P = C + Ke^{-rT} - S_0$$

In [37]:
def put_price_from_call(C,S0=100,K=103,T=1):
    discount = np.exp(-r * T)
    P = C + (K * discount) - S0
    return P

In [38]:
P = put_price_from_call(C=C,S0=S0,K=K,T=T)
round(P.item(),2)

12.43